In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules
import scipy.stats as stats
from statsmodels.stats import multitest as mt

df = pd.read_csv("C:\\Users\\Jens Harbers\\Documents\\schedenveg_reproduce.csv",sep=" ")

# manually remove some species, might be a different approach for rule mining in graslands
n = df.area.nunique()
cnt = df.variable.value_counts() / n

# --> For those wanting to remove additional species (Big 4 species and some quite abundand ones):
remove_species = ["FesPrat","LolPere","PhlPrat","PoaTriv","BroErec","TriFlav","PlaLanc"]
df = df[~df.variable.isin(remove_species)]
species_on_field = df[df.area == "A10_16"].variable.values


df = df[~df.variable.isin(cnt[(cnt <= 0.12)].index.values.tolist())]
df = pd.crosstab(df.area, df.variable)

frequent_itemsets = fpgrowth(df, min_support=0.12, use_colnames=True, max_len=20)

assoc = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.20, support_only=False)

# true if all species in antecedents are also existent in field

remove_cons = np.append(species_on_field,["RumAcet","DacGlom","FesPrat","LolPere","PhlPrat","PoaTriv","PoaPrat"])

assoc["antecedent_in_field"] = assoc.apply(lambda x: x.antecedents.issubset(species_on_field), axis=1)
assoc = assoc[assoc.antecedent_in_field == True]

# True if no species is listed in consequent
assoc["consequent_not_in_field"] = assoc.apply(lambda x: x.consequents.isdisjoint(remove_cons), axis=1)
assoc = assoc[assoc.consequent_not_in_field == True]

assoc["size_antecedents"] = assoc.apply(lambda x: len(x.antecedents), axis=1)
assoc["size_consequents"] = assoc.apply(lambda x: len(x.consequents), axis=1)


assoc['n1x'] = assoc['antecedent support'] * n
assoc['nx1'] = assoc['consequent support'] * n
assoc['n11'] = assoc['support'] * n

assoc['n0x'] = n - assoc['n1x']
assoc['nx0'] = n - assoc['nx1']

assoc['n10'] = assoc["n1x"] - assoc['n11']
assoc['n01'] = assoc["nx1"] - assoc['n11']
assoc['n00'] = assoc['n0x'] - assoc['n01']


# reduce calculations of Fisher's test (from 33.5M to 1550 calculations)
df2 = assoc[["n11", "n10", "n01", "n00"]].drop_duplicates().copy()
 
df2["pval"] = df2.apply(
    lambda x: stats.fisher_exact([[x.n11, x.n10], [x.n01, x.n00]], "greater")[1],
    axis=1
)
assoc = assoc.merge(df2)

assoc.iloc[:, 9:19] = assoc.iloc[:, 9:19].astype(int)
assoc = assoc.sort_values("pval")

assoc["bonferroni"] = mt.multipletests(
    assoc.pval, 0.05, method="bonferroni", is_sorted=True
)[1]

assoc["sidak"] = mt.multipletests(assoc.pval, 0.05, method="sidak", is_sorted=True)[1]

assoc["holm"] = mt.multipletests(assoc.pval, 0.05, method="holm", is_sorted=True)[1]

assoc["holm_sidak"] = mt.multipletests(
    assoc.pval, 0.05, method="holm-sidak", is_sorted=True
)[1]

assoc["simes_hochberg"] = mt.multipletests(
    assoc.pval, 0.05, method="simes-hochberg", is_sorted=True
)[1]

assoc["fdr_bh"] = mt.multipletests(
    assoc.pval, 0.05, method="fdr_bh", is_sorted=True
)[1]

assoc["fdr_tsbh"] = mt.multipletests(
    assoc.pval, 0.05, method="fdr_tsbh", is_sorted=True
)[1]

assoc["fdr_by"] = mt.multipletests(
    assoc.pval, 0.05, method="fdr_by", is_sorted=True
)[1]

assoc["fdr_tsbky"] = mt.multipletests(
    assoc.pval, 0.05, method="fdr_tsbky", is_sorted=True
)[1]

assoc["expected_species"] = assoc["size_consequents"] * assoc["confidence"]